# WORK IN PROGRESS

- 2 Content Based Models  

- First one has item Profile built with only tags column (+ Incorporate User ratings)
 - I will reach out to prof to figure out how to incorporate user ratings
 
- Second one has item Profile built with variety of columns (+ Incorporate User ratings)
 - User ratings incorporation should be the same for this as well
 
- If time permits, we can evaluate a final approach which uses global means (mean user rating, mean recipe rating)

In [92]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
# from dask_ml.feature_extraction import HashingVectorizer
import os
import seaborn as sns
from collections import Counter
import plotly.express as px
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import warnings
from scipy.sparse import csr_matrix
warnings.filterwarnings("ignore")

In [93]:
ratings = pd.read_csv('./clean_data/interactions_TRAIN.csv').drop('Unnamed: 0', axis=1)
recipes = pd.read_csv('./clean_data/recipes.csv').drop('Unnamed: 0', axis=1)

# ratings_TRAIN_dask = dd.from_pandas(ratings_TRAIN, 1).reset_index().drop('index', axis=1)
recipes_dask = dd.read_csv('./clean_data/recipes.csv').drop('Unnamed: 0', axis=1)

---

In [94]:
rated_recipes = ratings['recipe_id'].unique()

In [95]:
recipes = recipes[recipes['recipe_id'].isin(rated_recipes)]

---

In [96]:
ratings = ratings.merge(recipes, on = 'recipe_id')

In [97]:
ratings = ratings.drop(['date', 'minutes', 'contributor_id', 'submitted', 'n_steps', 'description', 'ingredients', 'n_ingredients', 'Calories', 'Total_fat_PDV', 'Sugar_PDV', 'Sodium_PDV', 'Protein_PDV', 'Saturated_fat_PDV', 'Carbohydrates_PDV', 'steps'], axis=1)

In [98]:
ratings['mean_user_rating'] = ratings.groupby('user_id')['rating'].transform('mean')
ratings['weighted_rating'] = ratings['rating'] - ratings['mean_user_rating']

In [99]:
ratings['tags'] = ratings['tags'].map(eval).map(lambda x: ' '.join(x))

In [100]:
ratings[ratings['user_id'] == 59801]

,user_id,recipe_id,rating,review,name,tags,mean_user_rating,weighted_rating
0,59801,8060,5,This really is a great basic meatball recipe. ...,carla s basic meatballs,60-minutes-or-less time-to-make course main-in...,4.458333,0.541667
5276,59801,25885,2,If I were rating this recipe on taste alone it...,banana banana bread,weeknight time-to-make course main-ingredient ...,4.458333,-2.458333
103420,59801,87075,5,This will become a staple at my house. Even m...,the blue mill tavern loosemeat sandwich,30-minutes-or-less time-to-make course main-in...,4.458333,0.541667
196600,59801,173742,5,The only way I do baked potatoes now. Nuff sa...,the best baked potatoes,time-to-make course main-ingredient preparatio...,4.458333,0.541667
203188,59801,88290,5,Way better than I expected!!! I made this thi...,apple or pear crisp for one,15-minutes-or-less time-to-make course main-in...,4.458333,0.541667
221776,59801,43508,4,Made this tonight and really enjoyed it. Even...,round steak withtomato gravy,weeknight time-to-make course main-ingredient ...,4.458333,-0.458333
235512,59801,115110,5,Fabulous!!! Both of my kids ate it. My oldes...,secretly delicious spinach pie,60-minutes-or-less time-to-make course main-in...,4.458333,0.541667
282091,59801,125399,5,I tried this using a basic homemade white brea...,french toast sticks oamc,60-minutes-or-less time-to-make course prepara...,4.458333,0.541667
297185,59801,40654,5,very good with minimal work. Thanks for posti...,tomato basil soup,60-minutes-or-less time-to-make course main-in...,4.458333,0.541667
330495,59801,64157,5,We just loved this one. Scrumpious!!!!! You ...,blackened chicken,15-minutes-or-less time-to-make main-ingredien...,4.458333,0.541667


In [110]:
tfidf = TfidfVectorizer(stop_words='english', token_pattern=r"(?u)\S\S+")
tfidf_matrix = tfidf.fit_transform(ratings['tags'])

In [ ]:
tfidf_matrix_arr = tfidf_matrix.toarray()

In [ ]:
user_profile = 0
for i, index in enumerate(ratings[ratings['user_id'] == 59801].index):
    user_profile += tfidf_matrix_arr[index] * weights[i]

In [108]:
user_profile

array([ 0.00000000e+00,  4.22074453e-02, -1.25895229e-01, -6.62554002e-02,
        1.08125227e-01, -7.71931916e-02, -9.34779609e-01,  0.00000000e+00,
        0.00000000e+00,  4.75655006e-02,  0.00000000e+00,  0.00000000e+00,
        5.43437292e-02, -3.63430855e-01,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  9.86299242e-02,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -1.63073578e-01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        8.59763106e-02,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [13]:
def generate_user_profile(user_id, ratings):
    
    ratings = ratings[ratings['user_id'] == user_id]
    tfidf = TfidfVectorizer(stop_words='english', token_pattern=r"(?u)\S\S+")
    tfidf_matrix = tfidf.fit_transform(ratings['tags'])
    weights = np.array(ratings['weighted_rating'])
    
    return weights * tfidf_matrix

In [14]:
generate_user_profile(906061, ratings).shape

(167,)

## Item Profiles based on TF-IDF of tags column

In [15]:
class TFIDF_Recommendations():
    
    def __init__(self, dataset):
        self.recipes = dataset
        self.recipes['tags'] = self.recipes['tags'].map(eval, meta = ('tags', 'object'))
        self.cosine_matrix = None
        self.flag = False
        self.indices = pd.Series(self.recipes.index, index = self.recipes['recipe_id']).drop_duplicates()

    
    def display_tag_distribution(self):
        
        tags_count = self.recipes['tags'].map(Counter).sum().compute().most_common()
        fig = px.bar(tags_count,
                     x = 0,
                     y = 1,
                     log_y = True,
                     title = "Count Distribution of Tags Column")
        fig.update_layout(
            xaxis_title="Tag Names",
            yaxis_title="Counts",
            yaxis = dict(
                tickmode = 'linear',
                tick0 = 0,
                dtick = 1
    ))
    
        return fig
    
    def display_cosine_matrix(self, n=5):
        
        try:
            fig = px.imshow(self.cosine_matrix[:n, :n],
                            labels = dict(x = "Recipe Index", y = "Recipe Index", color = "Cosine Similarity"),
                            title = "Cosine similarity of Recipes",
                            text_auto = True)
        except:
            raise Exception("Generate a cosine matrix first by calling generate_cosine_sim_matrix()")
            
        return fig
    
    def generate_user_profiles(self):
        pass
        

    
    
    
    def generate_cosine_sim_matrix(self):
            
            if not self.flag:
                temp = self.recipes['tags'].map(lambda x: ' '.join(x))
                tfidf = TfidfVectorizer(stop_words='english', token_pattern=r"(?u)\S\S+")
                tfidf_matrix = tfidf.fit_transform(temp)
                return tfidf, tfidf_matrix
                cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
                self.cosine_matrix = cosine_sim
                self.flag = True
        
    
    def generate_recommendations(self, recipe_id, n):
        
        idx = self.indices[recipe_id]
        sim_scores = list(enumerate(self.cosine_matrix[idx]))
        sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)[1:n+1]
        recipe_ids = [i[0] for i in sim_scores]
        
        return self.recipes.reset_index().loc[self.recipes.index.isin(recipe_ids)][['name', 'recipe_id']]
        

In [16]:
# model = TFIDF_Recommendations(recipes_dask)

In [17]:
# model = TFIDF_Recommendations(recipes)
# model.display_tag_distribution()